In [6]:
# import libraries

import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt


In [38]:
# read raw csv files
raw_seated_diners = pd.read_csv('input_data/YoY_Seated_Diner_Data.csv')

raw_state_cases = pd.read_csv('input_data/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv')

raw_country_cases = pd.read_csv('input_data/case_daily_trends__united_states.csv',skiprows =3)

raw_country_cases

,Date,New Cases,7-Day Moving Avg
0,Feb 9 2021,91034,107632
1,Feb 8 2021,86706,111329
2,Feb 7 2021,91762,116904
3,Feb 6 2021,107489,119906
4,Feb 5 2021,130261,124773
...,...,...,...
380,Jan 26 2020,3,0
381,Jan 25 2020,0,0
382,Jan 24 2020,1,0
383,Jan 23 2020,0,0


In [8]:
# create OpenTable country df
seated_diners_country = raw_seated_diners.loc[raw_seated_diners["Type"]=="country"]

seated_diners_country

,Type,Name,18-Feb,19-Feb,20-Feb,21-Feb,22-Feb,23-Feb,24-Feb,25-Feb,...,31-Jan,1-Feb,2-Feb,3-Feb,4-Feb,5-Feb,6-Feb,7-Feb,8-Feb,9-Feb
0,country,Global,-1.0,3.0,-1.0,-2.0,1.0,4.0,1.0,1.0,...,-36.78,-65.67,-65.19,-61.28,-57.32,-49.00,-47.01,-66.89,-60.98,-62.57
1,country,Australia,-3.0,-6.0,-3.0,-1.0,0.0,0.0,0.0,-2.0,...,83.31,39.50,49.74,61.33,72.21,77.40,95.85,100.09,72.52,56.08
2,country,Canada,-5.0,0.0,-3.0,-2.0,-1.0,8.0,-2.0,-3.0,...,-77.49,-83.83,-85.02,-83.07,-85.27,-79.97,-80.54,-80.18,-72.28,-76.22
3,country,Germany,11.0,4.0,3.0,5.0,1.0,2.0,11.0,4.0,...,-99.62,-99.75,-99.72,-99.67,-99.59,-99.42,-99.19,-99.75,-99.92,-99.48
4,country,Ireland,2.0,0.0,4.0,1.0,-3.0,4.0,-9.0,-8.0,...,-99.84,-100.00,-100.00,-100.00,-99.91,-99.83,-99.16,-99.93,-99.89,-99.98
5,country,Mexico,-5.0,-5.0,-4.0,-3.0,-1.0,2.0,-1.0,-3.0,...,-46.38,-54.53,-61.39,-64.40,-62.66,-57.15,-52.77,-68.10,-60.25,-60.22
6,country,United Kingdom,1.0,-1.0,-2.0,0.0,-1.0,-4.0,-6.0,-5.0,...,-99.76,-99.83,-99.83,-99.90,-99.88,-99.73,-99.63,-99.78,-99.81,-99.80
7,country,United States,0.0,4.0,0.0,-1.0,2.0,6.0,2.0,2.0,...,-23.52,-62.56,-61.23,-56.66,-51.62,-43.02,-41.57,-65.83,-57.92,-59.32


In [44]:
# filter to U.S. only

seated_diners_us = seated_diners_country.loc[seated_diners_country['Name']=='United States']

seated_diners_us_slim = seated_diners_us.drop(columns = ['Type','Name'])

# transpose the df to be joined

flipped_us = seated_diners_us_slim.transpose()

final_us = flipped_us.reset_index()

final_us.rename(columns={'index': 'Date', 7: 'Percent Change in Seated Diners'}, inplace=True)

final_us



0      18-Feb
1      19-Feb
2      20-Feb
3      21-Feb
4      22-Feb
        ...  
353     5-Feb
354     6-Feb
355     7-Feb
356     8-Feb
357     9-Feb
Name: Date, Length: 358, dtype: object

In [46]:
combined_df = pd.merge(raw_country_cases, final_us, left_index=True, right_index=True, how = 'outer')

combined_df

,Date_x,New Cases,7-Day Moving Avg,Date_y,Percent Change in Seated Diners
0,Feb 9 2021,91034,107632,18-Feb,0.0
1,Feb 8 2021,86706,111329,19-Feb,4.0
2,Feb 7 2021,91762,116904,20-Feb,0.0
3,Feb 6 2021,107489,119906,21-Feb,-1.0
4,Feb 5 2021,130261,124773,22-Feb,2.0
...,...,...,...,...,...
380,Jan 26 2020,3,0,NaN,NaN
381,Jan 25 2020,0,0,NaN,NaN
382,Jan 24 2020,1,0,NaN,NaN
383,Jan 23 2020,0,0,NaN,NaN


In [9]:
# create OpenTable states df
seated_diners_states = raw_seated_diners.loc[raw_seated_diners["Type"]=="state"]

seated_diners_states

,Type,Name,18-Feb,19-Feb,20-Feb,21-Feb,22-Feb,23-Feb,24-Feb,25-Feb,...,31-Jan,1-Feb,2-Feb,3-Feb,4-Feb,5-Feb,6-Feb,7-Feb,8-Feb,9-Feb
8,state,Alabama,-14.0,7.0,1.0,-2.0,8.0,2.0,1.0,0.0,...,10.07,-50.69,-44.34,-25.85,-32.04,-26.10,-26.13,-50.15,-45.14,-43.75
9,state,Alberta,2.0,10.0,2.0,1.0,5.0,0.0,-1.0,3.0,...,-99.33,-100.00,-99.67,-100.00,-99.60,-99.94,-99.95,-99.91,-13.10,-35.99
10,state,Arizona,0.0,1.0,8.0,4.0,-3.0,5.0,2.0,5.0,...,17.05,-50.66,-42.01,-41.44,-34.35,-26.01,-22.00,-56.54,-42.17,-42.37
11,state,Baden-Württemberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-99.59,-99.93,-99.95,-100.00,-99.88,-99.89,-99.91,-99.87,-100.00,-99.63
12,state,Baja California Sur,-6.0,-2.0,-2.0,-9.0,1.0,1.0,29.0,6.0,...,-29.52,-55.50,-51.40,-48.37,-37.30,-37.04,-30.24,-48.75,-40.44,-35.33
13,state,Bavaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-99.48,-99.89,-99.87,-99.56,-99.76,-99.56,-99.01,-100.00,-100.00,-99.56
14,state,British Columbia,8.0,-3.0,1.0,6.0,3.0,-2.0,-3.0,-5.0,...,-33.66,-46.51,-33.41,-32.93,-45.85,-34.35,-33.07,-37.62,-47.29,-46.19
15,state,California,-2.0,2.0,5.0,0.0,-2.0,7.0,3.0,6.0,...,-36.22,-73.53,-71.35,-70.57,-65.04,-55.70,-52.61,-69.86,-71.63,-71.68
16,state,Colorado,1.0,-8.0,-4.0,7.0,1.0,-8.0,-2.0,-4.0,...,31.93,-33.07,-30.90,-45.35,-36.98,-11.73,-27.69,-50.79,-43.58,-45.00
17,state,Connecticut,8.0,26.0,1.0,-3.0,-6.0,2.0,-2.0,6.0,...,-5.03,-94.18,-74.17,-54.14,-33.61,-32.07,-32.07,-79.47,-52.15,-67.93
